1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3. сделать разбиение набора данных не тренировочную и тестовую выборки
4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

In [127]:
from sklift.models import SoloModel, ClassTransformation, TwoModels
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds

from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [91]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/les4/data.csv')
data.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [92]:
data = data.rename(columns={'conversion': 'target',
                    'offer': 'treatment'}
            )
data.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [93]:
data.treatment = data.treatment.apply(lambda x: 0 if x in ['No Offer', 0] else 1)
data.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0


In [94]:
target = data['target'].values
data = data.drop(['target'], 1)
data.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment
0,10,142.44,1,0,Surburban,0,Phone,1
1,6,329.08,1,1,Rural,1,Web,0
2,7,180.65,0,1,Surburban,1,Web,1
3,9,675.83,1,0,Rural,1,Web,1
4,2,45.34,1,0,Urban,0,Web,1


In [95]:
data_dum = pd.get_dummies(data, columns=['zip_code', 'channel'])
data_dum

,recency,history,used_discount,used_bogo,is_referral,treatment,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,10,142.44,1,0,0,1,0,1,0,0,1,0
1,6,329.08,1,1,1,0,1,0,0,0,0,1
2,7,180.65,0,1,1,1,0,1,0,0,0,1
3,9,675.83,1,0,1,1,1,0,0,0,0,1
4,2,45.34,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,0,1,0,0,1,0,0,1
63996,5,38.91,0,1,1,1,0,0,1,0,1,0
63997,6,29.99,1,0,1,1,0,0,1,0,1,0
63998,1,552.94,1,0,1,1,0,1,0,1,0,0


In [96]:
X_train, X_test, y_train, y_test = train_test_split(data_dum, target, test_size=0.3) 
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((44800, 12), (19200, 12), (44800,), (19200,))

In [97]:
treat_train = X_train['treatment']
treat_test = X_test['treatment']
treat_train.shape, treat_test.shape

((44800,), (19200,))

In [123]:
# провести uplift-моделирование 3 способами:
# одна модель с признаком коммуникации (S learner),
# модель с трансформацией таргета (трансформация классов п. 2. 1)
# и вариант с двумя независимыми моделями
solo = SoloModel(
    CatBoostClassifier(iterations=20,
                       thread_count=2,
                       random_state=42,
                       silent=True))

trans = ClassTransformation(
    CatBoostClassifier(iterations=20,
                       thread_count=2,
                       random_state=42,
                       silent=True))

two = TwoModels(
    CatBoostClassifier(iterations=20,
                       thread_count=2,
                       random_state=42,
                       silent=True),
    RandomForestClassifier(n_estimators=300,
                           max_depth=5,
                           random_state=42))
model_list = [solo, trans, two]

In [125]:
res = pd.DataFrame(columns=['model', '10%', '20%', '30%'])
for model in model_list:
    model.fit(X_train, y_train, treat_train)
    pred = model.predict(X_test)
    res = res.append(
      {
          'model': f'{model.__class__.__name__}',
          '10%': uplift_at_k(y_true=y_test, uplift=pred, treatment=X_test['treatment'], strategy='by_group', k=0.1),
          '20%': uplift_at_k(y_true=y_test, uplift=pred, treatment=X_test['treatment'], strategy='by_group', k=0.2),
          '30%': uplift_at_k(y_true=y_test, uplift=pred, treatment=X_test['treatment'], strategy='by_group', k=0.3)
      },
      ignore_index=True
  )
res

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.


,model,10%,20%,30%
0,SoloModel,0.076267,0.069572,0.064526
1,ClassTransformation,0.243438,0.206325,0.177739
2,TwoModels,0.066261,0.053298,0.054201
